In [59]:
import sys
import os

project_root = os.path.abspath('..')
if project_root not in sys.path:
    sys.path.append(project_root)

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [60]:
import pandas as pd
import numpy as np

from src.data.make_dataset import procesar_incidencias
from src.features import build_features

import glob

ANIO_INICIO = 2017
ANIO_TERMINO = 2035

COLUMNAS_POBLACION_INE = [f"{i}" for i in range(ANIO_INICIO, ANIO_TERMINO + 1)]
COLUMNAS_POBLACION_FONASA = ["2018", "2019", "2020", "2021", "2022", "2023"]

COMUNAS_SSMO = [
    "Peñalolén",
    "Macul",
    "Ñuñoa",
    "La Reina",
    "Providencia",
    "Las Condes",
    "Vitacura",
    "Lo Barnechea",
    "Isla De Pascua",
]

COMUNAS_SSMC = ["Santiago", "Maipú", "Estación Central", "Cerrillos"]
COMUNAS_SSMS = [
    "San Bernardo",
    "Pedro Aguirre Cerda",
    "La Cisterna",
    "San Joaquín",
    "Lo Espejo",
    "San Miguel",
    "Paine",
    "Calera De Tango",
    "Buin",
    "El Bosque",
]

## Obtener poblacion INE y FONASA


In [61]:
df_ine = pd.read_csv("../data/processed/df_ine.csv")
df_fonasa = pd.read_csv("../data/processed/df_fonasa.csv", dtype={"ANO_INFORMACION": str})

## Definición de Estratos a Calcular

En este cuadernillo se quiere calcular la población para distintos Estratos y Grupos Etarios del País. Para esto, se utilizará la base de datos de la estimación de la población del Instituto Nacional de Estadísticas (INE) entre 2017 y 2035.

En primer lugar, se definirán los siguientes estratos a calcular:

1. País
2. Regiones del País
3. SSMO
4. SSMC
5. SSMS


### INE


In [62]:
ESTRATOS_A_CALCULAR_INE = {}

# Obtiene estratos a calcular
df_pais = df_ine.copy()
ESTRATOS_A_CALCULAR_INE["Pais"] = df_pais

for region in sorted(df_pais["M"].unique()):
    df_region = df_pais.query("M == @region").copy()
    ESTRATOS_A_CALCULAR_INE[region] = df_region

df_ssmo = df_pais[df_pais["Nombre Comuna"].isin(COMUNAS_SSMO)]
ESTRATOS_A_CALCULAR_INE["SSMO"] = df_ssmo

df_ssmc = df_pais[df_pais["Nombre Comuna"].isin(COMUNAS_SSMC)]
ESTRATOS_A_CALCULAR_INE["SSMC"] = df_ssmc

df_ssms = df_pais[df_pais["Nombre Comuna"].isin(COMUNAS_SSMS)]
ESTRATOS_A_CALCULAR_INE["SSMS"] = df_ssms

### FONASA


In [63]:
# Obtiene estratos a calcular FONASA
ESTRATOS_A_CALCULAR_FONASA = {}

# Estrato pais
df_pais_fonasa = df_fonasa.copy()
ESTRATOS_A_CALCULAR_FONASA["Pais"] = df_pais_fonasa

# Por region
for region in sorted(df_pais_fonasa["REGION"].fillna("").unique()):
    if region:
        df_region_fonasa = df_pais_fonasa.query("REGION == @region").copy()
        ESTRATOS_A_CALCULAR_FONASA[region] = df_region_fonasa

# Obtiene SSMO y SSMC
df_ssmo_fonasa = df_pais_fonasa[df_pais_fonasa["COMUNA"].isin(COMUNAS_SSMO)]
df_ssmc_fonasa = df_pais_fonasa[df_pais_fonasa["COMUNA"].isin(COMUNAS_SSMC)]
df_ssms_fonasa = df_pais_fonasa[df_pais_fonasa["COMUNA"].isin(COMUNAS_SSMS)]

ESTRATOS_A_CALCULAR_FONASA["SSMO"] = df_ssmo_fonasa
ESTRATOS_A_CALCULAR_FONASA["SSMC"] = df_ssmc_fonasa
ESTRATOS_A_CALCULAR_FONASA["SSMS"] = df_ssms_fonasa

## Definición de grupos etarios a calcular

En el próximo diccionario se definirán todos los grupos etarios que se quieren calcular para cada estrato previamente definidos.


In [64]:
# Define todos los grupos etarios a calcular para INE.
query_strings_ine = {
    "todos": "",  # Todo el pais
    "hombres": "hombre_mujer == 1",  # Hombres
    "mujeres": "hombre_mujer == 2",  # Mujeres
    "recien_nacidos_vivos": "Edad == 0",  # Recien Nacidos Vivos
    "sobre_15": "Edad >= 15",  # Sobre 15 anios
    "sobre_20": "Edad >= 20",  # Sobre 20 anios
    "sobre_45": "Edad >= 45",  # Sobre 45 anios
    "sobre_60": "Edad >= 60",  # Sobre 60 anios
    "sobre_65": "Edad >= 65",  # Sobre 65 anios
    "sobre_75": "Edad >= 75",  # Sobre 75 anios
    "entre_15_a_24_anios": "Edad >= 15 and Edad <= 24",  # Entre 15 y 24 anios
    "entre_25_a_44_anios": "Edad >= 25 and Edad <= 44",  # Entre 25 y 44 anios
    "entre_45_a_59_anios": "Edad >= 45 and Edad <= 59",  # Entre 45 y 59 anios
    "entre_60_y_70": "Edad >= 60 and Edad <= 70",  # Entre 60 y 70 anios
    "entre_60_a_74_anios": "Edad >= 60 and Edad <= 74",  # Entre 60 a 74 anios
}

# Define todos los grupos etarios a calcular para FONASA
query_strings_fonasa = {
    "todos": "",  # Todo el pais
    "hombres": "SEXO == 'HOMBRE'",  # Hombres
    "mujeres": "SEXO == 'MUJER'",  # Mujeres
    "recien_nacidos_vivos": "",  # Recien Nacidos Vivos
    "sobre_15": "EDAD_TRAMO >= 15",  # Sobre 15 anios
    "sobre_20": "EDAD_TRAMO >= 20",  # Sobre 20 anios
    "sobre_45": "EDAD_TRAMO >= 45",  # Sobre 45 anios
    "sobre_60": "EDAD_TRAMO >= 60",  # Sobre 60 anios
    "sobre_65": "EDAD_TRAMO >= 65",  # Sobre 65 anios
    "sobre_75": "EDAD_TRAMO >= 75",  # Sobre 75 anios
    "entre_15_a_24_anios": "EDAD_TRAMO >= 15 and EDAD_TRAMO < 25",  # Entre 15 y 24 anios
    "entre_25_a_44_anios": "EDAD_TRAMO >= 25 and EDAD_TRAMO < 45",  # Entre 25 y 44 anios
    "entre_45_a_59_anios": "EDAD_TRAMO >= 45 and EDAD_TRAMO < 60",  # Entre 45 y 59 anios
    "entre_60_y_70_anios": "EDAD_TRAMO >= 60 and EDAD_TRAMO <= 70",  # Entre 60 y 70 anios
    "entre_60_a_74_anios": "EDAD_TRAMO >= 60 and EDAD_TRAMO < 75",  # Entre 60 a 74 anios
    "mayores_a_75_anios": "EDAD_TRAMO >= 75",  # Mayores a 75 anios
}

Finalmente, se calculará la cantidad de población para distintos estratos y grupos etarios.


In [65]:
# Obtiene poblaciones INE por estratos y grupos etarios
poblaciones_ine = build_features.procesar_resultados_por_estrato_y_grupos_etarios(
    ESTRATOS_A_CALCULAR_INE, query_strings_ine, COLUMNAS_POBLACION_INE, "INE"
)

# Obtiene poblaciones FONASA por estratos y grupos etarios
poblaciones_fonasa = build_features.procesar_resultados_por_estrato_y_grupos_etarios(
    ESTRATOS_A_CALCULAR_FONASA, query_strings_fonasa, "CUENTA_BENEFICIARIOS", "FONASA"
)

## Obtener porcentaje de población FONASA total

En este caso, se quiere saber el % de población FONASA a nivel Nacional. Esto se utilizará para obtener los casos FONASA al 2035.


In [66]:
porcentaje_fonasa = build_features.obtener_porcentaje_de_fonasa_pais(
    poblaciones_ine,
    poblaciones_fonasa,
    COLUMNAS_POBLACION_FONASA,
)

grupos_de_columnas = ["Poblacion_INE", "Poblacion_FONASA", "Porcentaje_FONASA"]
columnas_nuevas = [
    f"{grupos_de_columnas[i // 7]}_{col}" for i, col in enumerate(porcentaje_fonasa.columns)
]

porcentaje_fonasa.columns = columnas_nuevas

In [67]:
print("Los porcentajes de FONASA son:")
display(porcentaje_fonasa)

Los porcentajes de FONASA son:


,Poblacion_INE_2018,Poblacion_INE_2019,Poblacion_INE_2020,Poblacion_INE_2021,Poblacion_INE_2022,Poblacion_INE_2023,Poblacion_INE_suma_poblacion,Poblacion_FONASA_2018,Poblacion_FONASA_2019,Poblacion_FONASA_2020,...,Poblacion_FONASA_2022,Poblacion_FONASA_2023,Poblacion_FONASA_suma_poblacion,Porcentaje_FONASA_2018,Porcentaje_FONASA_2019,Porcentaje_FONASA_2020,Porcentaje_FONASA_2021,Porcentaje_FONASA_2022,Porcentaje_FONASA_2023,Porcentaje_FONASA_porcentaje_fonasa_acumulado
Estrato,,,,,,,,,,,,,,,,,,,,,
Pais,18751405,19107216,19458310,19678363,19828563,19960889,116784746,13899613,14628760,14914359,...,15447864,16058254,89987476,0.741257,0.765614,0.766478,0.764221,0.779071,0.804486,0.770541
Antofagasta,645022,668563,691854,703746,709637,714142,4132964,413731,442613,457944,...,495329,527664,2806865,0.641422,0.662036,0.661908,0.667263,0.698003,0.738878,0.679141
Arica y Parinacota,241901,247036,252110,255380,257722,259802,1513951,189270,200890,205787,...,223720,232060,1264501,0.782428,0.813201,0.816259,0.833166,0.868067,0.893219,0.835232
Atacama,307835,311307,314709,316737,318004,319048,1887640,240935,252071,257377,...,268335,280270,1558916,0.782676,0.809718,0.817825,0.820643,0.843810,0.878457,0.825855
Aysén del General Carlos Ibáñez del Campo,106023,106680,107297,107737,108047,108306,644090,75617,79501,81125,...,82415,84083,484086,0.713213,0.745229,0.756079,0.755033,0.762770,0.776347,0.751581
Biobío,1645460,1654744,1663696,1670590,1676269,1681430,9992189,1331561,1373900,1391166,...,1419996,1461184,8373291,0.809233,0.830279,0.836190,0.835324,0.847117,0.869013,0.837984
Coquimbo,807213,821726,836096,848079,858769,869103,5040986,648137,683248,699935,...,725960,750485,4215546,0.802932,0.831479,0.837147,0.834570,0.845350,0.863517,0.836254
La Araucanía,1001420,1007965,1014343,1019548,1024029,1028201,6095506,844771,879389,899453,...,930651,957860,5420861,0.843573,0.872440,0.886735,0.891314,0.908813,0.931588,0.889321
Libertador General Bernardo O'Higgins,966486,978868,991063,1000959,1009552,1017701,5964629,773524,806650,826196,...,857620,887999,4982249,0.800347,0.824064,0.833646,0.829465,0.849506,0.872554,0.835299


En este apartado se leerán las incidencias, junto a las poblaciones respectivas donde se tienen que aplicar tales incidencias. Posteriormente, se calcularán los casos teóricos para cada problema de salud, y para cada estrato definido.


## Lectura de Incidencias por problema de salud


In [77]:
RUTA_PLANILLA_INCIDENCIAS = "../data/external/incidencias_y_prevalencias_INT.xlsx"
COLUMNAS_A_UTILIZAR = [
    "Diagnostico",
    "Diagnosticos Contenidos",
    "Macroproceso",
    "Estadística",
    "Casos (Cada 100.000)",
    "Edad Incidencia",
    "Área de Influencia Formal",
    "Área de Influencia Propuesta",
]

incidencias, limitados_por_oferta = procesar_incidencias(
    RUTA_PLANILLA_INCIDENCIAS, COLUMNAS_A_UTILIZAR
)

## Estimacion de casos con INE


In [78]:
# Une las incidencias con las poblacion calculadas segun los estratos respectivos
casos_INE = incidencias.merge(poblaciones_ine, how="left", on="Edad Incidencia")

# Multiplica la incidencia por las poblaciones
casos_INE[COLUMNAS_POBLACION_INE] = casos_INE[COLUMNAS_POBLACION_INE].multiply(
    casos_INE["rate_incidencia"], axis="index"
)

## Estimacion de casos FONASA, utilizando la poblacion INE y el % historico de la poblacion FONASA


In [79]:
# Copia los casos INE ya calculados
casos_FONASA_desde_INE = casos_INE.copy()

# Agrega el porcentaje FONASA de cada uno de los estratos calculados
casos_FONASA_desde_INE = casos_FONASA_desde_INE.merge(
    porcentaje_fonasa["Porcentaje_FONASA_2023"],
    how="left",
    left_on="Estrato",
    right_index=True,
)

# # Multiplica el % de FONASA con la poblacion INE
casos_FONASA_desde_INE[COLUMNAS_POBLACION_INE] = casos_FONASA_desde_INE[COLUMNAS_POBLACION_INE].mul(
    casos_FONASA_desde_INE["Porcentaje_FONASA_2023"], axis=0
)

## Acotación de casos según área de influencia para el INT

Una vez obtenido los casos proyectados hasta el 2035, se procederá a acotar cada uno de los diagnósticos y sus casos según su área de influencia determinada.


In [80]:
area_de_infl_INT = casos_FONASA_desde_INE.copy()

# Separa las areas de influencia por comas, para dejarlas en una lista
area_de_infl_INT["Área de Influencia Propuesta"] = area_de_infl_INT[
    "Área de Influencia Propuesta"
].str.split(",")
area_de_infl_INT = area_de_infl_INT.explode("Área de Influencia Propuesta")
area_de_infl_INT["Área de Influencia Propuesta"] = area_de_infl_INT[
    "Área de Influencia Propuesta"
].str.strip()

# Solamente deja las filas donde el Area de Influencia coincida con el Estrato
area_de_infl_INT = (
    area_de_infl_INT.query("`Área de Influencia Propuesta` == `Estrato`")
    .groupby("Diagnostico")[COLUMNAS_POBLACION_INE]
    .sum()
)
# Une el resultado de casos por area de influencia a las incidencias iniciales
area_de_infl_INT = (
    incidencias.set_index("Diagnostico")
    .merge(area_de_infl_INT, how="left", left_index=True, right_index=True)
    .reset_index()
)

# Elimina los diagnosticos acotados por oferta calculados incorrectamente
area_de_infl_INT = area_de_infl_INT.query("`Estadística` != 'Acotado por oferta'")

In [81]:
# Obtiene los casos de los diags acotado por oferta
area_de_infl_INT_acotados_por_oferta = limitados_por_oferta.copy()
for anio_ine in COLUMNAS_POBLACION_INE:
    area_de_infl_INT_acotados_por_oferta[anio_ine] = area_de_infl_INT_acotados_por_oferta[
        "Casos (Cada 100.000)"
    ]

In [82]:
# Une los casos acotados por oferta a los casos por incidencia y prevalencia
area_de_infl_INT = pd.concat([area_de_infl_INT, area_de_infl_INT_acotados_por_oferta])

## Obtención de resumen total para MINSAL

In [83]:
COLUMNAS_A_UNIR_PARA_RESUMEN = [
    "Diagnostico",
    "Diagnosticos Contenidos",
    "Macroproceso",
    "Estadística",
    "Casos (Cada 100.000)",
    "Edad Incidencia",
    "Área de Influencia Formal",
    "Área de Influencia Propuesta",
]

resumen_incidencias = incidencias.set_index(COLUMNAS_A_UNIR_PARA_RESUMEN)
resumen_poblacion_area_de_estudio = (
    incidencias.merge(poblaciones_ine, how="left", on="Edad Incidencia")
    .query("Estrato == 'Pais'")
    .set_index(COLUMNAS_A_UNIR_PARA_RESUMEN)["2035"]
)
resumen_poblacion_area_de_estudio.name = "poblacion_ine_2035_area_de_estudio"

resumen_casos_ine = casos_INE.query("Estrato == 'Pais'").set_index(COLUMNAS_A_UNIR_PARA_RESUMEN)[
    "2035"
]
resumen_casos_ine.name = "casos_teoricos_ine_2035_area_de_estudio"

porcentaje_fonasa_pais = porcentaje_fonasa["Porcentaje_FONASA_2023"]["Pais"]

resumen_casos_fonasa = casos_FONASA_desde_INE.query("Estrato == 'Pais'").set_index(
    COLUMNAS_A_UNIR_PARA_RESUMEN
)["2035"]
resumen_casos_fonasa.name = "casos_teoricos_fonasa_2035_area_de_estudio"

resumen_area_de_influencia = area_de_infl_INT.set_index(COLUMNAS_A_UNIR_PARA_RESUMEN)["2035"]
resumen_area_de_influencia.name = "casos_teoricos_fonasa_2035_area_de_influencia"

In [84]:
resumen_total_casos_teoricos = (
    resumen_incidencias.merge(
        resumen_poblacion_area_de_estudio, how="inner", left_index=True, right_index=True
    )
    .merge(resumen_casos_ine, how="inner", left_index=True, right_index=True)
    .merge(resumen_casos_fonasa, how="inner", left_index=True, right_index=True)
    .merge(resumen_area_de_influencia, how="right", left_index=True, right_index=True)
)

resumen_total_casos_teoricos["porcentaje_fonasa_pais"] = porcentaje_fonasa_pais
resumen_total_casos_teoricos = resumen_total_casos_teoricos.reset_index()

ORDEN_COLUMNAS_RESUMEN = [
    "Diagnostico",
    "Diagnosticos Contenidos",
    "Macroproceso",
    "Estadística",
    "Casos (Cada 100.000)",
    "Edad Incidencia",
    "poblacion_ine_2035_area_de_estudio",
    "casos_teoricos_ine_2035_area_de_estudio",
    "porcentaje_fonasa_pais",
    "casos_teoricos_fonasa_2035_area_de_estudio",
    "Área de Influencia Formal",
    "Área de Influencia Propuesta",
    "casos_teoricos_fonasa_2035_area_de_influencia",
]

resumen_total_casos_teoricos = resumen_total_casos_teoricos[ORDEN_COLUMNAS_RESUMEN]

In [85]:
dfs_a_guardar = {
    "poblacion_INE": poblaciones_ine,
    "poblacion_FONASA": poblaciones_fonasa,
    "porcentaje_FONASA_por_estrato": porcentaje_fonasa.reset_index(),
    "casos_teoricos_INE": casos_INE,
    "casos_teoricos_FONASA_desde_INE": casos_FONASA_desde_INE,
    "casos_area_de_influencia_INT": area_de_infl_INT,
    "resumen_total_INT": resumen_total_casos_teoricos,
}

with pd.ExcelWriter("../data/interim/casos_teoricos_diagnosticos.xlsx") as writer:
    for sheet_name, df_a_guardar in dfs_a_guardar.items():
        try:
            df_a_guardar = df_a_guardar.drop(columns="rate_incidencia")
        except KeyError:
            pass

        df_a_guardar.to_excel(writer, sheet_name=sheet_name, index=False)